In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
musics = pd.read_csv('musics.csv')

In [3]:
musics.dropna(inplace=True)

In [4]:
column_rename = {
    "the genre of the track": "genre",
    "Beats.Per.Minute -The tempo of the song": "bpm",
    "Energy- The energy of a song - the higher the value, the more energtic": "energy",
    "Danceability - The higher the value, the easier it is to dance to this song": "dance",
    "Loudness/dB - The higher the value, the louder the song": "loud",
    "Liveness - The higher the value, the more likely the song is a live recording": "live",
    "Valence - The higher the value, the more positive mood for the song": "valence",
    "Length - The duration of the song":"duration",
    "Acousticness - The higher the value the more acoustic the song is": "acoustic",
    "Speechiness - The higher the value the more spoken word the song contains": "speech",
    "Popularity- The higher the value the more popular the song is": "popularity"
}
musics = musics.rename(columns=column_rename)

In [5]:
remove_spaces = lambda x : x.replace(" ","").lower()
musics.artist = musics.artist.apply(remove_spaces)
musics.genre = musics.genre.apply(remove_spaces)

In [6]:
def set_values():
    keys = ['year','artist','genre','bpm','energy','dance','loud','live','valence','duration','acoustic','speech','popularity']
    musics['tags'] = ""
    for key in keys:
        musics[key] = musics[key].apply(lambda x : key+":"+str(x))
        musics.tags = musics.tags+musics[key]+" "
set_values()

In [7]:
data = musics[['title','artist','genre','tags']]

In [8]:
count_v = CountVectorizer(max_features=5000,stop_words='english')
vectors = count_v.fit_transform(data.tags).toarray()

In [9]:
similarity = cosine_similarity(vectors)

In [10]:
def recommend(music):
    index = data[data['title'] == music].index[0]
    distances = similarity[index]
    musics_with_index = list(enumerate(distances))
    musics_list = sorted(musics_with_index,reverse=True,key=lambda x:x[1])[1:6]
    for i in musics_list:
        print(data.iloc[i[0]].title)
recommend('Diamonds')

Work
We Found Love
American Oxygen
Part Of Me
Only Girl (In The World)


In [11]:
pickle.dump(data.to_dict(),open('data.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))